In [5]:
#!/usr/bin/env python
"""
Your task is to process the supplied file and use the csv module to extract data from it.
The data comes from NREL (National Renewable Energy Laboratory) website. Each file
contains information from one meteorological station, in particular - about amount of
solar and wind energy for each hour of day.

Note that the first line of the datafile is neither data entry, nor header. It is a line
describing the data source. You should extract the name of the station from it.

The data should be returned as a list of lists (not dictionaries).
You can use the csv modules "reader" method to get data in such format.
Another useful method is next() - to get the next line from the iterator.
You should only change the parse_file function.
"""
import csv
import os

DATADIR = ""
DATAFILE = "745090.csv"


def parse_file(datafile):
    name = ""
    data = []
    with open(datafile,'rb') as f:
        reader=csv.reader(f)
        name=reader.next()[1]
        reader.next()

        for row in reader:
            data.append(row)
    # Do not change the line below
    return (name, data)


def test():
    datafile = os.path.join(DATADIR, DATAFILE)
    name, data = parse_file(datafile)

    assert name == "MOUNTAIN VIEW MOFFETT FLD NAS"
    assert data[0][1] == "01:00"
    assert data[2][0] == "01/01/2005"
    assert data[2][5] == "2"


if __name__ == "__main__":
    test()

In [6]:
datafile = os.path.join(DATADIR, DATAFILE)
name, data = parse_file(datafile)
print name
print data[0][1]

MOUNTAIN VIEW MOFFETT FLD NAS
01:00


In [ ]:
# -*- coding: utf-8 -*-
'''
Find the time and value of max load for each of the regions
COAST, EAST, FAR_WEST, NORTH, NORTH_C, SOUTHERN, SOUTH_C, WEST
and write the result out in a csv file, using pipe character | as the delimiter.

An example output can be seen in the "example.csv" file.
'''

import xlrd
import os
import csv
from zipfile import ZipFile

datafile = "2013_ERCOT_Hourly_Load_Data.xls"
outfile = "2013_Max_Loads.csv"


def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
        myzip.extractall()


def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    data={}
    # YOUR CODE HERE
    for n in range (1, 9):
        station = sheet.cell_value(0, n)
        cv = sheet.col_values(n, start_rowx=1, end_rowx=None)

        maxval = max(cv)
        maxpos = cv.index(maxval) + 1
        maxtime = sheet.cell_value(maxpos, 0)
        realtime = xlrd.xldate_as_tuple(maxtime, 0)
        data[station] = {"maxval": maxval,
                         "maxtime": realtime}

    
    # Remember that you can use xlrd.xldate_as_tuple(sometime, 0) to convert
    # Excel date to Python tuple of (year, month, day, hour, minute, second)

    return data

def save_file(data, filename):
    # YOUR CODE HERE
    with open(filename, "w") as f:
        w = csv.writer(f, delimiter='|')
        w.writerow(["Station", "Year", "Month", "Day", "Hour", "Max Load"])
       
        for s in data:
           
            
            year, month, day, hour, _ , _= data[s]["maxtime"]
            print s, year, month, day, hour, data[s]["maxval"]
            w.writerow([s, year, month, day, hour, data[s]["maxval"]])
        
    
def test():
    open_zip(datafile)
    data = parse_file(datafile)
    save_file(data, outfile)
    
    number_of_rows = 0
    stations = []

    ans = {'FAR_WEST': {'Max Load': '2281.2722140000024',
                        'Year': '2013',
                        'Month': '6',
                        'Day': '26',
                        'Hour': '17'}}
    correct_stations = ['COAST', 'EAST', 'FAR_WEST', 'NORTH',
                        'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST']
    fields = ['Year', 'Month', 'Day', 'Hour', 'Max Load']

    with open(outfile) as of:
        csvfile = csv.DictReader(of, delimiter="|")
        for line in csvfile:
            station = line['Station']
            if station == 'FAR_WEST':
                for field in fields:
                    # Check if 'Max Load' is within .1 of answer
                    if field == 'Max Load':
                        max_answer = round(float(ans[station][field]), 1)
                        max_line = round(float(line[field]), 1)
                        assert max_answer == max_line

                    # Otherwise check for equality
                    else:
                        assert ans[station][field] == line[field]

            number_of_rows += 1
            stations.append(station)

        # Output should be 8 lines not including header
        assert number_of_rows == 8

        # Check Station Names
        assert set(stations) == set(correct_stations)

        
if __name__ == "__main__":
    test()


In [ ]:
#example.csv

Station|Year|Month|Day|Hour|Max Load
COAST|2013|01|01|10|12345.6
EAST|2013|01|01|10|12345.6
FAR_WEST|2013|01|01|10|12345.6
NORTH|2013|01|01|10|12345.6
NORTH_C|2013|01|01|10|12345.6
SOUTHERN|2013|01|01|10|12345.6
SOUTH_C|2013|01|01|10|12345.6
WEST|2013|01|01|10|12345.6

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
This exercise shows some important concepts that you should be aware about:
- using codecs module to write unicode files
- using authentication with web APIs
- using offset when accessing web APIs

To run this code locally you have to register at the NYTimes developer site 
and get your own API key. You will be able to complete this exercise in our UI
without doing so, as we have provided a sample result.

Your task is to process the saved file that represents the most popular
articles (by view count) from the last day, and return the following data:
- list of dictionaries, where the dictionary key is "section" and value is "title"
- list of URLs for all media entries with "format": "Standard Thumbnail"

All your changes should be in the article_overview function.
The rest of functions are provided for your convenience, if you want to access
the API by yourself.
"""
import json
import codecs
import requests

URL_MAIN = "http://api.nytimes.com/svc/"
URL_POPULAR = URL_MAIN + "mostpopular/v2/"
API_KEY = { "popular": "",
            "article": ""}


def get_from_file(kind, period):
    filename = "popular-{0}-{1}.json".format(kind, period)
    with open(filename, "r") as f:
        return json.loads(f.read())


def article_overview(kind, period):
    data = get_from_file(kind, period)
    titles = []
    urls =[]
    # YOUR CODE HERE
    for article in data:

        titles.append({article["section"]:article["title"]})
        if "media" in article:
                for m in article["media"]:
                    for mm in m["media-metadata"]:
                        if mm["format"] == "Standard Thumbnail":
                            urls.append(mm["url"])    
        
    return (titles, urls)


def query_site(url, target, offset):
    # This will set up the query with the API key and offset
    # Web services often use offset paramter to return data in small chunks
    # NYTimes returns 20 articles per request, if you want the next 20
    # You have to provide the offset parameter
    if API_KEY["popular"] == "" or API_KEY["article"] == "":
        print "You need to register for NYTimes Developer account to run this program."
        print "See Intructor notes for information"
        return False
    params = {"api-key": API_KEY[target], "offset": offset}
    r = requests.get(url, params = params)

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def get_popular(url, kind, days, section="all-sections", offset=0):
    # This function will construct the query according to the requirements of the site
    # and return the data, or print an error message if called incorrectly
    if days not in [1,7,30]:
        print "Time period can be 1,7, 30 days only"
        return False
    if kind not in ["viewed", "shared", "emailed"]:
        print "kind can be only one of viewed/shared/emailed"
        return False

    url += "most{0}/{1}/{2}.json".format(kind, section, days)
    data = query_site(url, "popular", offset)

    return data


def save_file(kind, period):
    # This will process all results, by calling the API repeatedly with supplied offset value,
    # combine the data and then write all results in a file.
    data = get_popular(URL_POPULAR, "viewed", 1)
    num_results = data["num_results"]
    full_data = []
    with codecs.open("popular-{0}-{1}.json".format(kind, period), encoding='utf-8', mode='w') as v:
        for offset in range(0, num_results, 20):        
            data = get_popular(URL_POPULAR, kind, period, offset=offset)
            full_data += data["results"]
        
        v.write(json.dumps(full_data, indent=2))


def test():
    titles, urls = article_overview("viewed", 1)
    assert len(titles) == 20
    assert len(urls) == 30
    assert titles[2] == {'Opinion': 'Professors, We Need You!'}
    assert urls[20] == 'http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-thumbStandard.jpg'


if __name__ == "__main__":
    test()

In [ ]:
#popular-viewed-1.json


[
  {
    "asset_id": 100000002711650.0, 
    "byline": "By ELI J. FINKEL", 
      "MARRIAGES", 
      "DIVORCE, SEPARATIONS AND ANNULMENTS", 
      "SOCIAL CONDITIONS AND TRENDS", 
      "SOCIOLOGY"
    ], 
    "title": "The All-or-Nothing Marriage", 
    "url": "http://www.nytimes.com/2014/02/15/opinion/sunday/the-all-or-nothing-marriage.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16COVER/16COVER-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16COVER/16COVER-superJumbo.jpg", 
            "width": 2048, 
            "height": 1552, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16COVER/16COVER-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16COVER/16COVER-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16COVER/16COVER-articleInline.jpg", 
            "width": 190, 
            "height": 144, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16COVER/16COVER-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16COVER/16COVER-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16COVER/16COVER-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Pieter Van Eenoge", 
        "caption": ""
      }, 
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16MARRIAGEJP/16MARRIAGEJP-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16MARRIAGEJP/16MARRIAGEJP-superJumbo.jpg", 
            "width": 2048, 
            "height": 1815, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16MARRIAGEJP/16MARRIAGEJP-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16MARRIAGEJP/16MARRIAGEJP-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16MARRIAGEJP/16MARRIAGEJP-articleInline.jpg", 
            "width": 190, 
            "height": 168, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16MARRIAGEJP/16MARRIAGEJP-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16MARRIAGEJP/16MARRIAGEJP-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16MARRIAGEJP/16MARRIAGEJP-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Pieter Van Eenoge", 
        "caption": ""
      }
    ], 
    "section": "Opinion", 
    "views": 1, 
    "id": "100000002711649", 
    "column": "Opinion", 
    "geo_facet": "", 
    "adx_keywords": "Marriages;Divorce, Separations and Annulments;Social Conditions and Trends;Sociology", 
    "source": "The New York Times", 
    "org_facet": "", 
    "published_date": "2014-02-16", 
    "per_facet": "", 
    "type": "Article", 
    "abstract": "Couples can be happier now than ever before. But it\u2019s rare."
  }, 
  {
    "asset_id": 100000002713490.0, 
    "byline": "By MAUREEN DOWD", 
    "des_facet": "", 
    "title": "Marry First, Then Cheat", 
    "url": "http://www.nytimes.com/2014/02/16/opinion/sunday/dowd-marry-first-then-cheat.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16DOWD/16DOWD-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16DOWD/16DOWD-superJumbo.jpg", 
            "width": 2048, 
            "height": 1364, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16DOWD/16DOWD-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16DOWD/16DOWD-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16DOWD/16DOWD-articleInline.jpg", 
            "width": 190, 
            "height": 127, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16DOWD/16DOWD-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16DOWD/16DOWD-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16DOWD/16DOWD-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Ed Alcock for The New York Times", 
        "caption": "At the Louvre on Friday, love was in the air, in contrast to the dark mood over a &ldquo;scandal.&rdquo;"
      }, 
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Dowd_New/Dowd_New-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Dowd_New/Dowd_New-articleInline.jpg", 
            "width": 190, 
            "height": 240, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Dowd_New/Dowd_New-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Fred R. Conrad/The New York Times", 
        "caption": ""
      }
    ], 
    "section": "Opinion", 
    "views": 2, 
    "id": "100000002713490", 
    "column": "Op-Ed Columnist", 
    "geo_facet": [
      "FRANCE"
    ], 
    "adx_keywords": "Hollande, Francois;Trierweiler, Valerie;Gayet, Julie;France", 
    "source": "The New York Times", 
    "org_facet": [
      "TWITTER", 
      "TWTR", 
      "NYSE"
    ], 
    "published_date": "2014-02-16", 
    "per_facet": [
      "HOLLANDE, FRANCOIS", 
      "TRIERWEILER, VALERIE", 
      "GAYET, JULIE"
    ], 
    "type": "Article", 
    "abstract": "In France, it\u2019s O.K. to cheat if you\u2019re married, but not if you\u2019re single."
  }, 
  {
    "asset_id": 100000002713490.0, 
    "byline": "By NICHOLAS KRISTOF", 
    "des_facet": [
      "COLLEGES AND UNIVERSITIES", 
      "UNITED STATES POLITICS AND GOVERNMENT", 
      "GRADUATE SCHOOLS AND STUDENTS"
    ], 
    "title": "Professors, We Need You!", 
    "url": "http://www.nytimes.com/2014/02/16/opinion/sunday/kristof-professors-we-need-you.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Kristof_New/Kristof_New-square320-v2.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Kristof_New/Kristof_New-thumbStandard-v4.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Kristof_New/Kristof_New-articleInline-v3.jpg", 
            "width": 190, 
            "height": 243, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Kristof_New/Kristof_New-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Kristof_New/Kristof_New-superJumbo.jpg", 
            "width": 1604, 
            "height": 2048, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Kristof_New/Kristof_New-square640-v2.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Kristof_New/Kristof_New-thumbLarge-v3.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Kristof_New/Kristof_New-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Damon Winter/The New York Times", 
        "caption": "Nicholas Kristof"
      }
    ], 
    "section": "Opinion", 
    "views": 3, 
    "id": "100000002713486", 
    "column": "Op-Ed Columnist", 
    "geo_facet": "", 
    "adx_keywords": "Colleges and Universities;United States Politics and Government;Graduate Schools and Students", 
    "source": "The New York Times", 
    "org_facet": [
      "FACEBOOK INC", 
      "FB", 
      "NASDAQ", 
      "TWITTER", 
      "TWTR", 
      "NYSE"
    ], 
    "published_date": "2014-02-16", 
    "per_facet": "", 
    "type": "Article", 
    "abstract": "Academics are some of the smartest minds in the world. So why are they making themselves irrelevant?"
  }, 
  {
    "asset_id": 100000002704900.0, 
    "byline": "By AL BAKER", 
    "des_facet": [
      "EDUCATION (K-12)", 
      "TESTS AND EXAMINATIONS"
    ], 
    "title": "Common Core Curriculum Now Has Critics on the Left", 
    "url": "http://www.nytimes.com/2014/02/17/nyregion/new-york-early-champion-of-common-core-standards-joins-critics.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/commoncore/commoncore-thumbStandard-v2.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/commoncore/commoncore-superJumbo-v2.jpg", 
            "width": 2048, 
            "height": 1347, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/commoncore/commoncore-sfSpan-v3.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/commoncore/commoncore-thumbLarge-v2.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/commoncore/commoncore-articleInline-v2.jpg", 
            "width": 190, 
            "height": 125, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/commoncore/commoncore-square640-v2.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/commoncore/commoncore-square320-v2.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/commoncore/commoncore-mediumThreeByTwo210-v2.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Victor J. Blue for The New York Times", 
        "caption": "Kimberly Celifarco with one of her kindergarten students, Gerson Perez, 5, at Public School 253 in Brooklyn."
      }, 
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE1/JP-COMMONCORE1-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE1/JP-COMMONCORE1-superJumbo.jpg", 
            "width": 2048, 
            "height": 1365, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE1/JP-COMMONCORE1-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE1/JP-COMMONCORE1-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE1/JP-COMMONCORE1-articleInline.jpg", 
            "width": 190, 
            "height": 127, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE1/JP-COMMONCORE1-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE1/JP-COMMONCORE1-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE1/JP-COMMONCORE1-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Nicole Bengiveno/The New York Times", 
        "caption": "From left, Carmen Mattimore, Kathleen Rusiecki, Evelyn Edwards and Nicole Senk attended a teacher training session last month on the Common Core at Public School 36 in the Bronx."
      }, 
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE2/JP-COMMONCORE2-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE2/JP-COMMONCORE2-superJumbo.jpg", 
            "width": 2048, 
            "height": 1365, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE2/JP-COMMONCORE2-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE2/JP-COMMONCORE2-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE2/JP-COMMONCORE2-articleInline.jpg", 
            "width": 190, 
            "height": 127, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE2/JP-COMMONCORE2-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE2/JP-COMMONCORE2-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE2/JP-COMMONCORE2-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Victor J. Blue for The New York Times", 
        "caption": ""
      }
    ], 
    "section": "N.Y. / Region", 
    "views": 4, 
    "id": "100000002704896", 
    "column": "", 
    "geo_facet": [
      "NEW YORK STATE"
    ], 
    "adx_keywords": "Education (K-12);Common Core;New York State;Tests and Examinations", 
    "source": "The New York Times", 
    "org_facet": "", 
    "published_date": "2014-02-17", 
    "per_facet": "", 
    "type": "Article", 
    "abstract": "The newest chorus of complaints about the common learning standards is coming from one of their earliest champions: New York State."
  }, 
  {
    "asset_id": 100000002712420.0, 
    "byline": "By ALEX WILLIAMS", 
    "des_facet": [
      "INTERNSHIPS", 
      "LABOR AND JOBS", 
      "HIRING AND PROMOTION"
    ], 
    "title": "For Interns, All Work and No Payoff", 
    "url": "http://www.nytimes.com/2014/02/16/fashion/millennials-internships.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16INTERNS_SPAN/16INTERNS_SPAN-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16INTERNS_SPAN/16INTERNS-superJumbo.jpg", 
            "width": 2048, 
            "height": 1335, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16INTERNS_SPAN/16INTERNS-sfSpan.jpg", 
            "width": 395, 
            "height": 257, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16INTERNS_SPAN/16INTERNS_SPAN-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16INTERNS_SPAN/16INTERNS-articleInline.jpg", 
            "width": 190, 
            "height": 124, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16INTERNS_SPAN/16INTERNS-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16INTERNS_SPAN/16INTERNS-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16INTERNS_SPAN/16INTERNS-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Michael Meister", 
        "caption": ""
      }, 
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS1/16jpinterns1-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS1/16jpinterns1-superJumbo.jpg", 
            "width": 2048, 
            "height": 1367, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS1/16jpinterns1-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS1/16jpinterns1-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS1/16jpinterns1-articleInline.jpg", 
            "width": 190, 
            "height": 127, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS1/16jpinterns1-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS1/16jpinterns1-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS1/16jpinterns1-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Karsten Moran for The New York Times", 
        "caption": "Breanne Thomas is interning for a tech company in New York."
      }, 
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS2/16JPINTERNS2-thumbStandard-v2.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS2/16JPINTERNS2-superJumbo-v2.jpg", 
            "width": 1385, 
            "height": 2048, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS2/16JPINTERNS2-sfSpan-v2.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS2/16JPINTERNS2-thumbLarge-v2.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS2/16JPINTERNS2-articleInline-v2.jpg", 
            "width": 190, 
            "height": 274, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS2/16JPINTERNS2-square640-v2.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS2/16JPINTERNS2-square320-v2.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS2/16JPINTERNS2-mediumThreeByTwo210-v2.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Stephanie Diani for The New York Times", 
        "caption": "Andrew Lang, in West Hollywood, Calif., is searching for a full-time job in the entertainment industry."
      }, 
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS3/16JPINTERNS3-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS3/16JPINTERNS3-superJumbo.jpg", 
            "width": 2048, 
            "height": 1465, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS3/16JPINTERNS3-sfSpan.jpg", 
            "width": 395, 
            "height": 264, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS3/16JPINTERNS3-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS3/16JPINTERNS3-articleInline.jpg", 
            "width": 190, 
            "height": 135, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS3/16JPINTERNS3-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS3/16JPINTERNS3-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/fashion/16JPINTERNS3/16JPINTERNS3-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Andrew Testa for The New York Times", 
        "caption": "Alec Dudson in the space in Manchester, England, where he edits Intern."
      }
    ], 
    "section": "Fashion & Style", 
    "views": 5, 
    "id": "100000002712417", 
    "column": "", 
    "geo_facet": "", 
    "adx_keywords": "Internships;Labor and Jobs;Colleges and Universities;Hiring and Promotion", 
    "source": "The New York Times", 
    "org_facet": [
      "GOOGLE INC", 
      "GOOG", 
      "NASDAQ", 
      "FACEBOOK INC", 
      "FB", 
      "NASDAQ", 
      "TWITTER", 
      "TWTR", 
      "NYSE"
    ], 
    "published_date": "2014-02-16", 
    "per_facet": "", 
    "type": "Article", 
    "abstract": "Unpaid internships are the bane of millennial workers with high aspirations."
  }, 
  {
    "asset_id": 100000002716010.0, 
    "byline": "By RICHARD SANDOMIR", 
    "des_facet": [
      "OLYMPIC GAMES (2014)", 
      "TELEVISION"
    ], 
    "title": "NBC Pushes Too Far in Bringing Bode Miller to Tears", 
    "url": "http://www.nytimes.com/2014/02/17/sports/olympics/nbc-pushes-too-far-in-bringing-bode-miller-to-tears.html", 
    "media": "", 
    "section": "Sports", 
    "views": 6, 
    "id": "100000002716012", 
    "column": "TV Sports", 
    "geo_facet": "", 
    "adx_keywords": "Olympic Games (2014);Television;National Broadcasting Co;Miller, Bode", 
    "source": "The New York Times", 
    "org_facet": [
      "NATIONAL BROADCASTING CO"
    ], 
    "published_date": "2014-02-17", 
    "per_facet": [
      "MILLER, BODE"
    ], 
    "type": "Article", 
    "abstract": "An NBC interviewer repeatedly questioned Miller about his brother, who died last year, after the skier tied Sunday for a bronze medal in the men\u2019s super-G."
  }, 
  {
    "asset_id": 100000002715840.0, 
    "byline": "By PAUL KRUGMAN", 
    "des_facet": [
      "ANTITRUST LAWS AND COMPETITION ISSUES", 
      "MERGERS, ACQUISITIONS AND DIVESTITURES", 
      "CABLE TELEVISION", 
      "COMPUTERS AND THE INTERNET"
    ], 
    "title": "Barons of Broadband", 
    "url": "http://www.nytimes.com/2014/02/17/opinion/krugman-barons-of-broadband.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Krugman_New/Krugman_New-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Krugman_New/Krugman_New-thumbStandard-v2.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Krugman_New/Krugman_New-articleInline-v2.jpg", 
            "width": 190, 
            "height": 285, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Krugman_New/Krugman_New-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Krugman_New/Krugman_New-superJumbo.jpg", 
            "width": 1366, 
            "height": 2048, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Krugman_New/Krugman_New-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Krugman_New/Krugman_New-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Fred R. Conrad/The New York Times", 
        "caption": "Paul Krugman"
      }
    ], 
    "section": "Opinion", 
    "views": 7, 
    "id": "100000002715844", 
    "column": "Op-Ed Columnist", 
    "geo_facet": [
      "UNITED STATES"
    ], 
    "adx_keywords": "Antitrust Laws and Competition Issues;Comcast Corporation;Time Warner Inc;Mergers, Acquisitions and Divestitures;Cable Television;Computers and the Internet;United States", 
    "source": "The New York Times", 
    "org_facet": [
      "COMCAST CORPORATION", 
      "TIME WARNER INC"
    ], 
    "published_date": "2014-02-17", 
    "per_facet": "", 
    "type": "Article", 
    "abstract": "Why did we stop worrying about monopoly power?"
  }, 
  {
    "asset_id": 100000002715400.0, 
    "byline": "By SAM DOLNICK", 
    "des_facet": [
      "OLYMPIC GAMES (2014)", 
      "SALT", 
      "ALPINE SKIING"
    ], 
    "title": "A Mad Dash for Salt Rescues Olympic Slopes", 
    "url": "http://www.nytimes.com/2014/02/17/sports/olympics/emergency-shipment-of-salt-averts-olympic-crisis.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt1/17salt1-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt1/17salt1-superJumbo.jpg", 
            "width": 2048, 
            "height": 1367, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt1/17salt1-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt1/17salt1-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt1/17salt1-articleInline.jpg", 
            "width": 190, 
            "height": 127, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt1/17salt1-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt1/17salt1-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt1/17salt1-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Dmitry Lovetsky/Associated Press", 
        "caption": "A worker threw salt to prepare the track during the men's 4x10-kilometer cross-country relay on Sunday."
      }, 
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-2/17salt-2-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-2/17salt-2-superJumbo.jpg", 
            "width": 2048, 
            "height": 1365, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-2/17salt-2-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-2/17salt-2-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-2/17salt-2-articleInline.jpg", 
            "width": 190, 
            "height": 127, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-2/17salt-2-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-2/17salt-2-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-2/17salt-2-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Doug Mills/The New York Times", 
        "caption": "Alpine experts use salt to overcome soft snow conditions when a hard, icy surface is preferable. The salt melts the soft snow, and when the temperature drops &mdash; usually overnight &mdash; a layer of ice forms."
      }, 
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-1/17salt-1-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-1/17salt-1-superJumbo.jpg", 
            "width": 2048, 
            "height": 1328, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-1/17salt-1-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-1/17salt-1-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-1/17salt-1-articleInline.jpg", 
            "width": 190, 
            "height": 123, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-1/17salt-1-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-1/17salt-1-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/17salt-1/17salt-1-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Ezra Shaw/Getty Images", 
        "caption": "A course marshal throws&nbsp;big-grained salt onto the snow before the women's super-G event on Saturday."
      }
    ], 
    "section": "Sports", 
    "views": 8, 
    "id": "100000002715400", 
    "column": "", 
    "geo_facet": "", 
    "adx_keywords": "Olympic Games (2014);Salt;Alpine Skiing", 
    "source": "The New York Times", 
    "org_facet": "", 
    "published_date": "2014-02-17", 
    "per_facet": "", 
    "type": "Article", 
    "abstract": "Soft snow threatened Alpine events at the Sochi Games, but a hastily arranged purchase from Switzerland of 24 tons of a large-grain variety of salt unavailable in Russia saved the day."
  }, 
  {
    "asset_id": 100000002714130.0, 
    "byline": "By JAMES RISEN and LAURA POITRAS", 
    "des_facet": [
      "INTERNATIONAL TRADE AND WORLD MARKET", 
      "SURVEILLANCE OF CITIZENS BY GOVERNMENT"
    ], 
    "title": "Spying by N.S.A. Ally Entangled U.S. Law Firm", 
    "url": "http://www.nytimes.com/2014/02/16/us/eavesdropping-ensnared-american-law-firm.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA1/jpNSA1-thumbStandard-v3.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA1/jpNSA1-superJumbo-v2.jpg", 
            "width": 1050, 
            "height": 772, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA1/jpNSA1-sfSpan-v3.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA1/jpNSA1-thumbLarge-v3.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA1/jpNSA1-articleInline-v2.jpg", 
            "width": 190, 
            "height": 140, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA1/jpNSA1-square640-v3.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA1/jpNSA1-square320-v3.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA1/jpNSA1-mediumThreeByTwo210-v3.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Stephen Crowley/The New York Times", 
        "caption": "The Indonesian Embassy in Washington, left, and the building where Mayer Brown has an office. Indonesia retained the American law firm for help in trade talks."
      }, 
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA2/jpNSA2-thumbStandard-v2.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA2/jpNSA2-superJumbo-v2.jpg", 
            "width": 1050, 
            "height": 951, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA2/jpNSA2-sfSpan-v2.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA2/jpNSA2-thumbLarge-v2.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA2/jpNSA2-articleInline-v2.jpg", 
            "width": 190, 
            "height": 172, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA2/jpNSA2-square640-v2.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA2/jpNSA2-square320-v2.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/us/jpNSA2/jpNSA2-mediumThreeByTwo210-v2.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Aman Rahman/Agence France-Presse \u2014 Getty Images", 
        "caption": "Indonesian women at a factory making clove cigarettes, the subject of a trade dispute with the United States."
      }
    ], 
    "section": "U.S.", 
    "views": 9, 
    "id": "100000002714131", 
    "column": "", 
    "geo_facet": [
      "INDONESIA"
    ], 
    "adx_keywords": "National Security Agency;International Trade and World Market;Snowden, Edward J;Surveillance of Citizens by Government;Espionage and Intelligence Services;Wiretapping and Other Eavesdropping Devices and Methods;Indonesia;Mayer Brown;Australian Signals Directorate", 
    "source": "The New York Times", 
    "org_facet": [
      "NATIONAL SECURITY AGENCY", 
      "MAYER BROWN", 
      "AUSTRALIAN SIGNALS DIRECTORATE"
    ], 
    "published_date": "2014-02-16", 
    "per_facet": [
      "SNOWDEN, EDWARD J"
    ], 
    "type": "Article", 
    "abstract": "A document obtained by Edward J. Snowden shows that Australia consulted with the N.S.A. about monitoring an American law firm that was representing Indonesia in trade disputes with the United States."
  }, 
  {
    "asset_id": 100000002710440.0, 
    "byline": "By GINIA BELLAFANTE", 
    "des_facet": [
      "DRUG ABUSE AND TRAFFIC"
    ], 
    "title": "Addicted on Staten Island", 
    "url": "http://www.nytimes.com/2014/02/16/nyregion/addicted-to-drugs-on-staten-island.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/nyregion/16BIGCITY_SPAN/16BIGCITY_SPAN-thumbStandard-v2.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/nyregion/16BIGCITY_SPAN/16BIGCITY_SPAN-superJumbo-v2.jpg", 
            "width": 2048, 
            "height": 1366, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/nyregion/16BIGCITY_SPAN/16BIGCITY_SPAN-sfSpan-v2.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/nyregion/16BIGCITY_SPAN/16BIGCITY_SPAN-thumbLarge-v2.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/nyregion/16BIGCITY_SPAN/16BIGCITY_SPAN-articleInline-v2.jpg", 
            "width": 190, 
            "height": 127, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/nyregion/16BIGCITY_SPAN/16BIGCITY_SPAN-square640-v2.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/nyregion/16BIGCITY_SPAN/16BIGCITY_SPAN-square320-v2.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/nyregion/16BIGCITY_SPAN/16BIGCITY_SPAN-mediumThreeByTwo210-v2.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Robert Stolarik for The New York Times", 
        "caption": "Rebecca Ann Wolfson is now free of drugs, after more than 15 failed attempts at rehabilitation."
      }
    ], 
    "section": "N.Y. / Region", 
    "views": 10, 
    "id": "100000002710440", 
    "column": "Big City", 
    "geo_facet": [
      "STATEN ISLAND (NYC)"
    ], 
    "adx_keywords": "Drug Abuse and Traffic;Staten Island (NYC)", 
    "source": "The New York Times", 
    "org_facet": "", 
    "published_date": "2014-02-16", 
    "per_facet": "", 
    "type": "Article", 
    "abstract": "New patterns of drug usage remind us that it is not marijuana that ought to be feared as a pathway to more insidious forms of addiction; it is substances that remain perfectly legal."
  }, 
  {
    "asset_id": 100000002715850.0, 
    "byline": "By DANNY HAKIM", 
    "des_facet": [
      "REAL ESTATE AND HOUSING (RESIDENTIAL)"
    ], 
    "title": "Saving an Endangered British Species: The Pub", 
    "url": "http://www.nytimes.com/2014/02/17/business/international/saving-an-endangered-british-species-the-pub.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/business/pubs/pubs-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/business/pubs/pubs-superJumbo.jpg", 
            "width": 2048, 
            "height": 1374, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/business/pubs/pubs-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/business/pubs/pubs-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/business/pubs/pubs-articleInline.jpg", 
            "width": 190, 
            "height": 127, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/business/pubs/pubs-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/business/pubs/pubs-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/business/pubs/pubs-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Adam Ferguson for The New York Times", 
        "caption": "The Old White Bear pub in London closed this month, and its future is uncertain."
      }
    ], 
    "section": "Business Day", 
    "views": 11, 
    "id": "100000002715846", 
    "column": "", 
    "geo_facet": [
      "LONDON (ENGLAND)", 
      "GREAT BRITAIN"
    ], 
    "adx_keywords": "Bars and Nightclubs;Legislatures and Parliaments;London (England);Great Britain;Real Estate and Housing (Residential)", 
    "source": "The New York Times", 
    "org_facet": "", 
    "published_date": "2014-02-17", 
    "per_facet": "", 
    "type": "Article", 
    "abstract": "Changing economics and tastes in Britain have claimed roughly one out of every five \u201clocals\u201d in the last 20 years."
  }, 
  {
    "asset_id": 100000002715610.0, 
    "byline": "By JER\u00c9 LONGMAN", 
    "des_facet": [
      "OLYMPIC GAMES (2014)", 
      "FIGURE SKATING"
    ], 
    "title": "Behind Meryl Davis and Charlie White, U.S. Is Close to Its First Ice Dance Gold", 
    "url": "http://www.nytimes.com/2014/02/17/sports/olympics/behind-meryl-davis-and-charlie-white-us-is-close-to-its-first-ice-dance-gold.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-superJumbo.jpg", 
            "width": 2048, 
            "height": 1363, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-articleInline.jpg", 
            "width": 190, 
            "height": 126, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "James Hill for The New York Times", 
        "caption": "Performing to music from &ldquo;My Fair Lady,&rdquo; Meryl Davis and Charlie White scored a world record of 78.89 points."
      }
    ], 
    "section": "Sports", 
    "views": 12, 
    "id": "100000002715610", 
    "column": "", 
    "geo_facet": [
      "UNITED STATES", 
      "CANADA"
    ], 
    "adx_keywords": "Olympic Games (2014);Figure Skating;Davis, Meryl;White, Charlie;Virtue, Tessa;Moir, Scott;United States;Canada", 
    "source": "The New York Times", 
    "org_facet": "", 
    "published_date": "2014-02-17", 
    "per_facet": [
      "DAVIS, MERYL", 
      "WHITE, CHARLIE", 
      "VIRTUE, TESSA", 
      "MOIR, SCOTT"
    ], 
    "type": "Article", 
    "abstract": "In the short program, Davis and White moved a step closer \u2014 a fox trot and quickstep closer, to be precise \u2014 to becoming the first American Olympic champions in ice dance."
  }, 
  {
    "asset_id": 100000002711220.0, 
    "byline": "By NATASHA SINGER", 
    "des_facet": [
      "ROBOTS AND ROBOTICS"
    ], 
    "title": "Intel\u2019s Sharp-Eyed Social Scientist", 
    "url": "http://www.nytimes.com/2014/02/16/technology/intels-sharp-eyed-social-scientist.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/technology/16bell-personal-billboard/16bell-personal-billboard-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/technology/16bell-personal-billboard/16bell-personal-billboard-superJumbo.jpg", 
            "width": 1618, 
            "height": 1079, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/technology/16bell-personal-billboard/16bell-personal-billboard-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/technology/16bell-personal-billboard/16bell-personal-billboard-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/technology/16bell-personal-billboard/16bell-personal-billboard-articleInline.jpg", 
            "width": 190, 
            "height": 127, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/technology/16bell-personal-billboard/16bell-personal-billboard-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/technology/16bell-personal-billboard/16bell-personal-billboard-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/technology/16bell-personal-billboard/16bell-personal-billboard-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "", 
        "caption": "In a company video, Intel showed examples of its \"Personal Billboard\" program, which lets people post photos and text messages of up to 140 characters on exterior laptop screens with the goal of encouraging real-time communication."
      }
    ], 
    "section": "Technology", 
    "views": 13, 
    "id": "100000002711216", 
    "column": "", 
    "geo_facet": "", 
    "adx_keywords": "Bell, Genevieve;Intel Corporation;Anthropology;Research;Robots and Robotics;Wearable Computing", 
    "source": "The New York Times", 
    "org_facet": [
      "INTEL CORPORATION"
    ], 
    "published_date": "2014-02-16", 
    "per_facet": [
      "BELL, GENEVIEVE"
    ], 
    "type": "Article", 
    "abstract": "As part of Intel\u2019s mission to be regarded as more than just a chip maker, the anthropologist Genevieve Bell leads a globe-trotting team that is trying to learn what people crave in their electronics."
  }, 
  {
    "asset_id": 100000002712320.0, 
    "byline": "", 
    "des_facet": [
      "FIGURE SKATING", 
      "OLYMPIC GAMES (2014)"
    ], 
    "title": "Hanyu Falls Twice, but Still Wins Gold", 
    "url": "http://www.nytimes.com/interactive/2014/02/14/sports/olympics/mens-figure-skating.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/14/sports/olympics/mens-figure-skating-1392420419784/mens-figure-skating-1392420419784-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/14/sports/olympics/mens-figure-skating-1392420419784/mens-figure-skating-1392420419784-articleInline.jpg", 
            "width": 190, 
            "height": 91, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/14/sports/olympics/mens-figure-skating-1392420419784/mens-figure-skating-1392420419784-superJumbo.jpg", 
            "width": 875, 
            "height": 418, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/14/sports/olympics/mens-figure-skating-1392420419784/mens-figure-skating-1392420419784-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/14/sports/olympics/mens-figure-skating-1392420419784/mens-figure-skating-1392420419784-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/14/sports/olympics/mens-figure-skating-1392420419784/mens-figure-skating-1392420419784-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/14/sports/olympics/mens-figure-skating-1392420419784/mens-figure-skating-1392420419784-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "", 
        "type": "image", 
        "copyright": "", 
        "caption": ""
      }
    ], 
    "section": "Sports", 
    "views": 14, 
    "id": "100000002712320", 
    "column": "", 
    "geo_facet": "", 
    "adx_keywords": "", 
    "source": "The New York Times", 
    "org_facet": "", 
    "published_date": "2014-02-14", 
    "per_facet": [
      "HANYU, YUZURU", 
      "CHAN, PATRICK"
    ], 
    "type": "Interactive", 
    "abstract": "After a world record-setting short program performance, Yuzuru Hanyu, a teenager from Japan, fell twice in his free skate but still prevailed."
  }, 
  {
    "asset_id": 100000002712160.0, 
    "byline": "By EDWARD FRENKEL", 
    "des_facet": [
      "MATHEMATICS", 
      "PHILOSOPHY"
    ], 
    "title": "Is the Universe a Simulation?", 
    "url": "http://www.nytimes.com/2014/02/16/opinion/sunday/is-the-universe-a-simulation.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16GRAY/16GRAY-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16GRAY/16GRAY-superJumbo.jpg", 
            "width": 1716, 
            "height": 2048, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16GRAY/16GRAY-sfSpan-v2.jpg", 
            "width": 395, 
            "height": 472, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16GRAY/16GRAY-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16GRAY/16GRAY-articleInline-v2.jpg", 
            "width": 190, 
            "height": 182, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16GRAY/16GRAY-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16GRAY/16GRAY-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16GRAY/16GRAY-mediumThreeByTwo210-v2.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Olimpia Zagnoli", 
        "caption": ""
      }
    ], 
    "section": "Opinion", 
    "views": 15, 
    "id": "100000002712155", 
    "column": "Gray Matter", 
    "geo_facet": "", 
    "adx_keywords": "Mathematics;Philosophy", 
    "source": "The New York Times", 
    "org_facet": "", 
    "published_date": "2014-02-16", 
    "per_facet": "", 
    "type": "Article", 
    "abstract": "If so, that would help explain some mysterious things about math."
  }, 
  {
    "asset_id": 100000002713470.0, 
    "byline": "By ROSS DOUTHAT", 
    "des_facet": [
      "PARENTING", 
      "SOCIAL CONDITIONS AND TRENDS"
    ], 
    "title": "Parental Pity Party", 
    "url": "http://www.nytimes.com/2014/02/16/opinion/sunday/douthat-parental-pity-party.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Douthat_New/Douthat_New-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Douthat_New/Douthat_New-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Douthat_New/Douthat_New-articleInline.jpg", 
            "width": 190, 
            "height": 285, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Douthat_New/Douthat_New-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Douthat_New/Douthat_New-superJumbo.jpg", 
            "width": 1366, 
            "height": 2048, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Douthat_New/Douthat_New-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2010/09/16/opinion/Douthat_New/Douthat_New-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Josh Haner/The New York Times", 
        "caption": "Ross Douthat"
      }
    ], 
    "section": "Opinion", 
    "views": 16, 
    "id": "100000002713472", 
    "column": "Op-Ed Columnist", 
    "geo_facet": "", 
    "adx_keywords": "Parenting;All Joy and No Fun: The Paradox of Modern Parenthood (Book);Social Conditions and Trends", 
    "source": "The New York Times", 
    "org_facet": "", 
    "published_date": "2014-02-16", 
    "per_facet": "", 
    "type": "Article", 
    "abstract": "First comes the baby, and then all the whining starts about how impossible it all is."
  }, 
  {
    "asset_id": 100000002704110.0, 
    "byline": "", 
    "des_facet": [
      "BARS AND NIGHTCLUBS"
    ], 
    "title": "Behind Bars", 
    "url": "http://www.nytimes.com/interactive/2014/02/16/opinion/sunday/20140216_SCHOTTBARWORDS.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/opinion/sunday/20140216_SCHOTTBARWORDS-1392239127937/20140216_SCHOTTBARWORDS-1392239127937-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/opinion/sunday/20140216_SCHOTTBARWORDS-1392239127937/20140216_SCHOTTBARWORDS-1392239127937-articleInline.jpg", 
            "width": 190, 
            "height": 127, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/opinion/sunday/20140216_SCHOTTBARWORDS-1392239127937/20140216_SCHOTTBARWORDS-1392239127937-superJumbo.jpg", 
            "width": 300, 
            "height": 200, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/opinion/sunday/20140216_SCHOTTBARWORDS-1392239127937/20140216_SCHOTTBARWORDS-1392239127937-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/opinion/sunday/20140216_SCHOTTBARWORDS-1392239127937/20140216_SCHOTTBARWORDS-1392239127937-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "", 
        "type": "image", 
        "copyright": "", 
        "caption": ""
      }
    ], 
    "section": "Opinion", 
    "views": 17, 
    "id": "100000002704108", 
    "column": "Op-Chart | Ben Schott", 
    "geo_facet": "", 
    "adx_keywords": "", 
    "source": "The New York Times", 
    "org_facet": "", 
    "published_date": "2014-02-16", 
    "per_facet": [
      "SCHOTT, BEN"
    ], 
    "type": "Interactive", 
    "abstract": "The secret vocabulary of New York\u2019s finest drinking establishments."
  }, 
  {
    "asset_id": 100000002713340.0, 
    "byline": "By TIM LAHEY", 
    "des_facet": [
      "HOSPITALS"
    ], 
    "title": "A Watchful Eye in Hospitals", 
    "url": "http://www.nytimes.com/2014/02/17/opinion/a-watchful-eye-in-hospitals.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/opinion/0217OPEDwebb/0217OPEDwebb-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/opinion/0217OPEDwebb/0217OPEDwebb-superJumbo.jpg", 
            "width": 2048, 
            "height": 1417, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/opinion/0217OPEDwebb/0217OPEDwebb-sfSpan.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/opinion/0217OPEDwebb/0217OPEDwebb-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/opinion/0217OPEDwebb/0217OPEDwebb-articleInline.jpg", 
            "width": 190, 
            "height": 131, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/opinion/0217OPEDwebb/0217OPEDwebb-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/opinion/0217OPEDwebb/0217OPEDwebb-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/17/opinion/0217OPEDwebb/0217OPEDwebb-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Neil Webb", 
        "caption": ""
      }
    ], 
    "section": "Opinion", 
    "views": 18, 
    "id": "100000002713340", 
    "column": "Op-Ed Contributor", 
    "geo_facet": [
      "UNITED STATES"
    ], 
    "adx_keywords": "Hospitals;Privacy;Cameras;Ethics (Institutional);United States", 
    "source": "The New York Times", 
    "org_facet": "", 
    "published_date": "2014-02-17", 
    "per_facet": "", 
    "type": "Article", 
    "abstract": "Are hidden cameras a violation of patients\u2019 right to privacy?"
  }, 
  {
    "asset_id": 100000002704910.0, 
    "byline": "By JOHN ANDERSON", 
    "des_facet": [
      "MOVIES"
    ], 
    "title": "The Greatest Film Role Ever Cast", 
    "url": "http://www.nytimes.com/2014/02/16/movies/heading-off-criticism-when-picking-an-actor-to-play-jesus.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS2/16MOVIEJESUS2-thumbStandard-v2.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS2/16MOVIE-JESUS3-superJumbo.jpg", 
            "width": 2048, 
            "height": 1366, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS2/16MOVIE-JESUS3-sfSpan.jpg", 
            "width": 395, 
            "height": 264, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS2/16MOVIEJESUS2-thumbLarge-v2.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS2/16MOVIE-JESUS3-articleInline.jpg", 
            "width": 190, 
            "height": 127, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS2/16MOVIEJESUS2-square640-v2.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS2/16MOVIEJESUS2-square320-v2.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS2/16MOVIE-JESUS3-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Casey Crafford/20th Century Fox", 
        "caption": "The Portuguese actor Diogo Morgado as Jesus in the new &ldquo;Son of God.&rdquo; He is the latest film actor to play the role, one going back to 1898. Mr. Morgado is more fair-skinned than historians might call for, but he could &ldquo;portray the lion and the lamb,&rdquo; one of the producers said."
      }, 
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS1COMBO_SPAN/16MOVIEJESUS1-thumbStandard.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS1COMBO_SPAN/16MOVIEJESUS1-superJumbo.jpg", 
            "width": 2048, 
            "height": 978, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS1COMBO_SPAN/16MOVIEJESUS1-sfSpan.jpg", 
            "width": 395, 
            "height": 189, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS1COMBO_SPAN/16MOVIEJESUS1-thumbLarge.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS1COMBO_SPAN/16MOVIEJESUS1-articleInline.jpg", 
            "width": 190, 
            "height": 91, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS1COMBO_SPAN/16MOVIEJESUS1-square640.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS1COMBO_SPAN/16MOVIEJESUS1-square320.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/arts/16MOVIEJESUS1COMBO_SPAN/16MOVIEJESUS1COMBO_SPAN-mediumThreeByTwo210.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Photofest; Reuters; Universal Pictures", 
        "caption": "The many faces of a cinematic Jesus, from left, Max von Sydow in &ldquo;The Greatest Story Ever Told&rdquo; (1965); Jeffrey Hunter in &ldquo;King of Kings&rdquo; (1961); and Willem Dafoe in &ldquo;The Last Temptation of Christ&rdquo; (1988)."
      }
    ], 
    "section": "Movies", 
    "views": 19, 
    "id": "100000002704914", 
    "column": "", 
    "geo_facet": "", 
    "adx_keywords": "Movies;Jesus Christ;Son of God (Movie);Morgado, Diogo", 
    "source": "The New York Times", 
    "org_facet": [
      "TWITTER", 
      "TWTR", 
      "NYSE"
    ], 
    "published_date": "2014-02-16", 
    "per_facet": [
      "JESUS CHRIST", 
      "MORGADO, DIOGO"
    ], 
    "type": "Article", 
    "abstract": "Filmmakers try to find a Jesus who is a solemn soul, matinee idol, conflicted Messiah and now a hottie on Twitter."
  }, 
  {
    "asset_id": 100000002713340.0, 
    "byline": "By FARHAD MANJOO", 
    "des_facet": [
      "CABLE TELEVISION", 
      "COMPUTERS AND THE INTERNET", 
      "MERGERS, ACQUISITIONS AND DIVESTITURES"
    ], 
    "title": "Comcast vs. the Cord Cutters", 
    "url": "http://www.nytimes.com/2014/02/16/business/media/comcast-vs-the-cord-cutters.html", 
    "media": [
      {
        "media-metadata": [
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/business/16-CONSUMER/16-CONSUMER-thumbStandard-v2.jpg", 
            "width": 75, 
            "height": 75, 
            "format": "Standard Thumbnail"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/business/16-CONSUMER/16-CONSUMER-superJumbo.jpg", 
            "width": 1226, 
            "height": 1661, 
            "format": "superJumbo"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/business/16-CONSUMER/16-CONSUMER-sfSpan-v3.jpg", 
            "width": 395, 
            "height": 263, 
            "format": "Large"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/business/16-CONSUMER/16-CONSUMER-thumbLarge-v2.jpg", 
            "width": 150, 
            "height": 150, 
            "format": "thumbLarge"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/business/16-CONSUMER/16-CONSUMER-articleInline.jpg", 
            "width": 190, 
            "height": 257, 
            "format": "Normal"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/business/16-CONSUMER/16-CONSUMER-square640-v2.jpg", 
            "width": 640, 
            "height": 640, 
            "format": "square640"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/business/16-CONSUMER/16-CONSUMER-square320-v2.jpg", 
            "width": 320, 
            "height": 320, 
            "format": "square320"
          }, 
          {
            "url": "http://graphics8.nytimes.com/images/2014/02/16/business/16-CONSUMER/16-CONSUMER-mediumThreeByTwo210-v2.jpg", 
            "width": 210, 
            "height": 140, 
            "format": "mediumThreeByTwo210"
          }
        ], 
        "subtype": "photo", 
        "type": "image", 
        "copyright": "Minh Uong/The New York Times", 
        "caption": ""
      }
    ], 
    "section": "Business Day", 
    "views": 20, 
    "id": "100000002713336", 
    "column": "News Analysis", 
    "geo_facet": "", 
    "adx_keywords": "Comcast Corporation;Time Warner Cable Inc;Cable Television;Computers and the Internet;Mergers, Acquisitions and Divestitures;Prices (Fares, Fees and Rates)", 
    "source": "The New York Times", 
    "org_facet": [
      "COMCAST CORPORATION", 
      "TIME WARNER CABLE INC"
    ], 
    "published_date": "2014-02-16", 
    "per_facet": "", 
    "type": "Article", 
    "abstract": "Comcast\u2019s deal to acquire Time Warner Cable won\u2019t shake the industry\u2019s game plan: Keep viewers wedded to cable."
  }
]